# Advertisement
Пример отчета по расчету аудитории рекламных кампаний.

## Описание задачи

Посчитаем аудиторию по бренду в разбивке на рекламные позиции.

Общие параметры:
- Период: 3 - 9 января 2022
- География: Россия 0+
- Население: Возрастная группа 12-34
- Типы пользования интернетом: Web Desktop, App Mobile

Статистики:
- Reach (reach)
- Reach% (reachPer)
- OTS (ots)
- Reach N+ (reachN)
- Universe (uni)


# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

### Общие параметры для задания

Для начала зададим общие параметры

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2022-01-03', '2022-01-09')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,3]

### Получим ID Бренда
Для построения отчета необходимо получить идентификатор бренда 

Для этого воспользуемся методами поиска в справочнике по рекламе. Работа со справочником по рекламе подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

In [ ]:
cats.get_brand(brand='Введите название бренда')

## Задание

Перейдем к формированию задания.


### Расчет аудитории по рекламным позициям

In [ ]:
# Задаем название для отображения в DataFrame
project_name = 'Brand_Name'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии
demo_filter = 'ageGroup IN (1,2,3)'

# Задаем фильтр по бренду, в нашем случае это ID бренда: 
mart_filter = 'brandId = BRANDID'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["advertisementId", "advertisementName"]

# Указываем список статистик для расчета
statistics = ["reach", "reachPer", "ots", "reachN", "uni"]

scales = {'reachN':[(2, 4), (5,10)]}


# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('ad', project_name, date_filter, usetype_filter, geo_filter,
                                      demo_filter, mart_filter, slices, statistics, scales)


# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_task(task_json))

# Получаем результат
df_ad = mtask.result2table(mtask.get_result(task), project_name = project_name)
df_ad

## Сохраняем в Excel

In [ ]:
df_info = mtask.get_report_info()

with pd.ExcelWriter(mtask.get_excel_filename('audience-brand')) as writer:
    df_ad.to_excel(writer, sheet_name='Report', index=False)
    df_info.to_excel(writer, sheet_name='Info', index=False)